In [339]:
import pandas as pd
import numpy as np

In [340]:
df = pd.read_csv("../data/processed/data_merged.csv")

In [341]:
df.head()

,MARITAL,RENTHOM1,VETERAN3,EMPLOY1,_AGEG5YR,_INCOMG1,_EDUCAG,_CHLDCNT,_EXTETH3,HHADULT,...,_PHYS14D,_MENT14D,_BMI5CAT,LSATISFY,social_vulnerability_score,tobacco_use,health_risk_score,disability_score,trauma_exposure_score,gun_risk_score
0,1.0,1.0,2.0,7.0,13.0,9.0,4.0,1.0,9.0,NaN,...,1.0,1.0,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1,3.0,1.0,2.0,2.0,13.0,3.0,2.0,1.0,9.0,NaN,...,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,2.0,7.0,8.0,6.0,4.0,1.0,9.0,NaN,...,2.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,2.0,7.0,14.0,9.0,2.0,1.0,9.0,NaN,...,1.0,1.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0
4,1.0,1.0,2.0,5.0,5.0,3.0,3.0,1.0,9.0,NaN,...,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [342]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 36 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   MARITAL                     445124 non-null  float64
 1   RENTHOM1                    445123 non-null  float64
 2   VETERAN3                    440959 non-null  float64
 3   EMPLOY1                     438936 non-null  float64
 4   _AGEG5YR                    445132 non-null  float64
 5   _INCOMG1                    445132 non-null  float64
 6   _EDUCAG                     445132 non-null  float64
 7   _CHLDCNT                    445131 non-null  float64
 8   _EXTETH3                    445132 non-null  float64
 9   HHADULT                     349071 non-null  float64
 10  SEXVAR                      445132 non-null  float64
 11  MEDCOST1                    445128 non-null  float64
 12  _HLTHPLN                    445132 non-null  float64
 13  _AIDTST4      

In [343]:
duplicates = df.duplicated().sum()

df = df.drop_duplicates()

In [344]:
# Columnas a eliminar por amplia presencia de nulos
cols_to_drop = ["CDSOCIAL", "CDDISCUS", "CRGVREL4", "CRGVLNG1", "CRGVHRS1", "MARIJAN1"]
df = df.drop(cols_to_drop, axis=1)

In [345]:
df = df.dropna()

In [346]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182940 entries, 1211 to 445131
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   MARITAL                     182940 non-null  float64
 1   RENTHOM1                    182940 non-null  float64
 2   VETERAN3                    182940 non-null  float64
 3   EMPLOY1                     182940 non-null  float64
 4   _AGEG5YR                    182940 non-null  float64
 5   _INCOMG1                    182940 non-null  float64
 6   _EDUCAG                     182940 non-null  float64
 7   _CHLDCNT                    182940 non-null  float64
 8   _EXTETH3                    182940 non-null  float64
 9   HHADULT                     182940 non-null  float64
 10  SEXVAR                      182940 non-null  float64
 11  MEDCOST1                    182940 non-null  float64
 12  _HLTHPLN                    182940 non-null  float64
 13  _AIDTST4        

## Recodificacion de variables

In [347]:
#MARITAL
def clasificar_estado_civil(valor):
    if valor == 1:
        return "Casado"
    elif valor == 2:
        return "Divorciado"
    elif valor == 3:
        return "Viudo"
    elif valor in [4, 5, 6]:
        return "Soltero"
    else:
        return None  # Para 9 Refused o valores faltantes
df['estado_civil'] = df['MARITAL'].apply(clasificar_estado_civil)
df = df[df['estado_civil'].notna()]
df = df.drop('MARITAL', axis=1)
df["estado_civil"].value_counts()

estado_civil
Casado        96888
Soltero       47847
Divorciado    24601
Viudo         12483
Name: count, dtype: int64

In [348]:
def clasificar_vivienda(valor):
    if valor == 1:
        return "Propia"
    elif valor == 2:
        return "Alquilado"
    elif valor in [3, 7, 9]:
        return "Otro"
    else:
        return None 
    
df['situacion_vivienda'] = df['RENTHOM1'].apply(clasificar_vivienda)

# Eliminar filas con valores faltantes
df = df[df['situacion_vivienda'].notna()]
df = df.drop('RENTHOM1', axis=1)
df['situacion_vivienda'].value_counts()

situacion_vivienda
Propia       124346
Alquilado     47017
Otro          10456
Name: count, dtype: int64

In [349]:
#VETEREN1
def clasificar_servicio_militar(valor):
    if valor == 1:
        return 0 #"Sí"
    elif valor == 2:
        return 1 #"No"
    else:
        return None
df['servicio_militar'] = df['VETERAN3'].apply(clasificar_servicio_militar)
df = df[df['servicio_militar'].notna()]
df = df.drop('VETERAN3', axis=1)
print(df['servicio_militar'].value_counts())

servicio_militar
1.0    160296
0.0     21268
Name: count, dtype: int64


In [350]:
#EMPLO1
def clasificar_situacion_laboral(valor):
    if valor == 1:
        return "Empleado"
    elif valor == 2:
        return "Autónomo"
    elif valor in [3, 4, 5, 6, 8, 9]:
        return "Desempleado"
    elif valor == 7:
        return "Jubilado"
    else:
        return None
df['situacion_laboral'] = df['EMPLOY1'].apply(clasificar_situacion_laboral)
df = df[df['situacion_laboral'].notna()]
df = df.drop('EMPLOY1', axis=1)
print(df['situacion_laboral'].value_counts())

situacion_laboral
Empleado       88631
Jubilado       43099
Desempleado    31879
Autónomo       17955
Name: count, dtype: int64


In [351]:
#EDAD
def edad(valor):
    if valor in [1, 2, 3]:  # 18-24, 25-29, 30-34
        return "18-35"
    elif valor in [4, 5, 6, 7, 8, 9]:  # 35-39, 40-44, 45-49, 50-54, 55-59, 60-64
        return "36-65"
    elif valor in [10, 11, 12, 13]:  # 65-69, 70-74, 75-79, 80+
        return "66+"
    else:
        return None

df['edad'] = df['_AGEG5YR'].apply(edad)
df = df[df['edad'].notna()]
df = df.drop('_AGEG5YR', axis=1)
print(df['edad'].value_counts())

edad
36-65    94678
66+      49805
18-35    35382
Name: count, dtype: int64


In [352]:
#_EDUCAG
def educacion(row):
    if row in [2, 4]:
        return 0 #'Se graduó'
    elif row in [1, 3] or pd.isna(row):
        return 1 #'No estudió / No concluyó'
    else:
        return 'Otro'
df['educacion'] = df['_EDUCAG'].apply(educacion)
df = df.drop('_EDUCAG', axis=1)
df['educacion'].value_counts()

educacion
0       119414
1        60121
Otro       330
Name: count, dtype: int64

In [353]:
def niños_hogar(valor):
    if valor == 1:
        return 0  # Sin niños
    elif valor in [2, 3, 4]:
        return 1  # De 1-3 niños
    elif valor in [5, 6]:
        return 2  # +3 de niños
    else:
        return -1
    
df['child_home'] = df['_CHLDCNT'].apply(niños_hogar)
df = df.drop('_CHLDCNT', axis=1)
df['child_home'].value_counts()

child_home
 0    125748
 1     48839
 2      4052
-1      1226
Name: count, dtype: int64

In [354]:
#Incomg1

def ingresos(valor):
    if valor == 1:
        return -2
    elif valor in [2, 3]:    # 15mil - 35mil
        return 0
    elif valor in [4, 5]:    # 36mil - 100mil
        return 1
    elif valor in [6, 7]:    # 100mil o más
        return 2
    else:  # No sabe cuanto
        return -1
df['ingresos_anuales'] = df['_INCOMG1'].apply(ingresos)
df = df.drop('_INCOMG1', axis=1)
df['ingresos_anuales'].value_counts()

ingresos_anuales
 1    66410
 2    47069
 0    30397
-1    26700
-2     9289
Name: count, dtype: int64

In [355]:
df = df.rename(columns={"_RFBING6": "heavy_drinker"})

In [356]:
# Question:  Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
# 9 (Don’t know/Refused/Missing) >> Cambiar a -1
df.loc[df["_TOTINDA"] == 9, "_TOTINDA"] = -1
df.loc[df["_TOTINDA"] == 2, "_TOTINDA"] = 0

print(df["_TOTINDA"].value_counts())
df = df.rename(columns={"_TOTINDA": "act_fisica"})

_TOTINDA
 1.0    140213
 0.0     39320
-1.0       332
Name: count, dtype: int64


In [357]:
# Question:  (Ever told) (you had) a depressive disorder (including depression, major depression, dysthymia, or minor depression)
# Eliminamos NAN y los valores 7 y 9 (Don’t know/Refused/Missing)
df['ADDEPEV3'].dropna()
df = df[~df["ADDEPEV3"].isin([7, 9])]
df.loc[df["ADDEPEV3"] == 2, "ADDEPEV3"] = 0
print(df["ADDEPEV3"].value_counts())

ADDEPEV3
0.0    137722
1.0     41165
Name: count, dtype: int64


In [358]:
# Question:  Has a doctor, nurse, or other health professional ever told you that you tested positive for COVID 19?
# df['COVIDPOS'].value_counts(dropna=False)

df.loc[df["COVIDPOS"] == 3, "COVIDPOS"] = 1
df.loc[df["COVIDPOS"] == 2, "COVIDPOS"] = 0
df.loc[(df["COVIDPOS"] == 7) | (df["COVIDPOS"] == 9), "COVIDPOS"] = -1
df = df.rename(columns={"COVIDPOS": "covid_pos"})
print(df["covid_pos"].value_counts())

covid_pos
 0.0    116065
 1.0     62367
-1.0       455
Name: count, dtype: int64


In [359]:
df.loc[df["_PHYS14D"] == 1, "_PHYS14D"] = 0
df.loc[df["_PHYS14D"] == 2, "_PHYS14D"] = 1
df.loc[df["_PHYS14D"] == 3, "_PHYS14D"] = 2
df.loc[df["_PHYS14D"] == 9, "_PHYS14D"] = -1

df = df.rename(columns={"_PHYS14D": "salud_fisica"})

df["salud_fisica"].value_counts()

salud_fisica
 0.0    106557
 1.0     45991
 2.0     22774
-1.0      3565
Name: count, dtype: int64

In [360]:
df.loc[df["_MENT14D"] == 1, "_MENT14D"] = 0
df.loc[df["_MENT14D"] == 2, "_MENT14D"] = 1
df.loc[df["_MENT14D"] == 3, "_MENT14D"] = 2
df.loc[df["_MENT14D"] == 9, "_MENT14D"] = -1

df = df.rename(columns={"_MENT14D": "salud_mental"})

df["salud_mental"].value_counts()

salud_mental
 0.0    101585
 1.0     48242
 2.0     26097
-1.0      2963
Name: count, dtype: int64

In [361]:
df.loc[df["_BMI5CAT"] == 1, "_BMI5CAT"] = 0
df.loc[df["_BMI5CAT"] == 2, "_BMI5CAT"] = 1
df.loc[df["_BMI5CAT"] == 3, "_BMI5CAT"] = 2
df.loc[df["_BMI5CAT"] == 4, "_BMI5CAT"] = 3

df = df.rename(columns={"_BMI5CAT": "index_peso"})
df["index_peso"].value_counts()

index_peso
2.0    63280
3.0    62145
1.0    50656
0.0     2806
Name: count, dtype: int64

In [362]:
df.loc[(df["LSATISFY"] == 1) | (df["LSATISFY"] == 2), "LSATISFY"] = 0
df.loc[(df["LSATISFY"] == 3) | (df["LSATISFY"] == 4), "LSATISFY"] = 1
df.loc[(df["LSATISFY"] == 7) | (df["LSATISFY"] == 9), "LSATISFY"] = -1

df = df.rename(columns={"LSATISFY": "life_satisfaction"})
df["life_satisfaction"].value_counts()

life_satisfaction
 0.0    167103
 1.0     10178
-1.0      1606
Name: count, dtype: int64

In [363]:
df = df.drop(["_EXTETH3", "HHADULT"], axis=1)

In [364]:
df = df.rename(columns={"SEXVAR": "gender"})
df["gender"].value_counts() # 1: Male, 2: Female

gender
1.0    91494
2.0    87393
Name: count, dtype: int64

In [365]:
# 1: Yes, 2: No >> Cambiar a 1: Yes, 0: No
df.loc[df["MEDCOST1"] > 2, "MEDCOST1"] = -1
df.loc[df["MEDCOST1"] == 2, "MEDCOST1"] = 0
df = df.rename(columns={"MEDCOST1": "missed_care_cost"})
df.missed_care_cost.value_counts()

missed_care_cost
 0.0    161618
 1.0     16775
-1.0       494
Name: count, dtype: int64

In [366]:
# 1: Yes, 2: No >> Cambiar a 0: Yes, 1: No
df.loc[df["_HLTHPLN"] > 2, "_HLTHPLN"] = -1
df.loc[df["_HLTHPLN"] == 1, "_HLTHPLN"] = 0
df.loc[df["_HLTHPLN"] == 2, "_HLTHPLN"] = 1

df = df.rename(columns={"_HLTHPLN": "insurance"})
df.insurance.value_counts()

insurance
 0.0    163050
 1.0     10470
-1.0      5367
Name: count, dtype: int64

In [367]:
# 1: Yes, 2: No >> Cambiar a 1: Yes, 0: No
df.loc[df["_AIDTST4"] > 2, "_AIDTST4"] = -1
df.loc[df["_AIDTST4"] == 2, "_AIDTST4"] = 0
df = df.rename(columns={"_AIDTST4": "test_hiv"})
df.test_hiv.value_counts()

test_hiv
 0.0    106084
 1.0     64458
-1.0      8345
Name: count, dtype: int64

In [368]:
# 1: Yes, 2: No >> Cambiar a 1: Yes, 0: No
df.loc[df["HIVRISK5"] > 2, "HIVRISK5"] = -1
df.loc[df["HIVRISK5"] == 2, "HIVRISK5"] = 0
df = df.rename(columns={"HIVRISK5": "drugs_prostit"})
df.drugs_prostit.value_counts()

drugs_prostit
 0.0    168856
 1.0      9294
-1.0       737
Name: count, dtype: int64

In [369]:
df.loc[df["SLEPTIM1"] > 24, "SLEPTIM1"] = -1
df.loc[(df["SLEPTIM1"] > 0) & (df["SLEPTIM1"] <= 5), "SLEPTIM1"] = 2
df.loc[(df["SLEPTIM1"] > 5) & (df["SLEPTIM1"] <= 9), "SLEPTIM1"] = 0
df.loc[(df["SLEPTIM1"] > 9) & (df["SLEPTIM1"] <= 24), "SLEPTIM1"] = 1
df = df.rename(columns={"SLEPTIM1": "sleep_time"})
df.sleep_time.value_counts()

sleep_time
 0.0    152160
 2.0     19666
 1.0      5546
-1.0      1515
Name: count, dtype: int64

In [370]:
df.columns

Index(['gender', 'missed_care_cost', 'insurance', 'test_hiv', 'sleep_time',
       'drugs_prostit', 'act_fisica', 'heavy_drinker', 'ADDEPEV3', 'covid_pos',
       'salud_fisica', 'salud_mental', 'index_peso', 'life_satisfaction',
       'social_vulnerability_score', 'tobacco_use', 'health_risk_score',
       'disability_score', 'trauma_exposure_score', 'gun_risk_score',
       'estado_civil', 'situacion_vivienda', 'servicio_militar',
       'situacion_laboral', 'edad', 'educacion', 'child_home',
       'ingresos_anuales'],
      dtype='object')

In [371]:
df.to_csv("../data/processed/data_ready.csv", index=False)